In [7]:
# Install necessary libraries within the Notebook environment
# !pip install selenium beutifulsoup4 webdriver-manager

f1_teams에 새로운 url을 넣고 코드를 실행하면,
</br>기존에 동일한 이름으로 존재하던 파일을 대체하는 게 아니라
</br>기존 내용에 새로운 링크에서 크롤링한 내용이 추가됨.

In [8]:
import os
import re
import time
import collections
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

# Global Configuration
SOURCE_NAME = "data"
OUTPUT_DIR = f"(ENG)F1_{SOURCE_NAME}"

def setup_driver():
    """Initializes the Selenium WebDriver with Headless Chrome options."""
    chrome_options = Options()
    # Run in headless mode for stability in Notebook environments (no UI window)
    chrome_options.add_argument("--headless") 
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    # Set a common user-agent to avoid basic bot detection
    chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")
    chrome_options.add_argument("--window-size=1920,1080")
    
    # Automatically manage ChromeDriver installation
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=chrome_options)
    return driver

In [ ]:
# Input Dictionary: F1 Teams and their URLs
# 'all' key contains general links to be analyzed and sorted.
f1_teams = {
    "all": [
    ],
    "Scuderia_Ferrari": [
    ],
    "Red_Bull_Racing": [
    ],
    "McLaren": [
    ],
    "Haas_F1_Team": [
    ],
    "Sauber_Motorsport": [
    ],
    "Aston_Martin_in_Formula_One": [
    ],
    "Mercedes-Benz_in_Formula_One": [
    ],
    "Williams_Racing": [
    ],
    "Racing_Bulls": [
    ],
    "Alpine_F1_Team": [
        ]
}

In [ ]:
CRAWL_LIST = [
    ("Aston Martin", "2020", "https://www.motorsport.com/f1/news/?filters%5Bdiscovery_type%5D%5B%5D=1&filters%5Bchampionship%5D=2871&filters%5Bteam%5D%5B%5D=97"),
    ("Aston Martin", "2020", "https://www.motorsport.com/f1/news/?filters%5Bdiscovery_type%5D%5B%5D=1&filters%5Bchampionship%5D=2871&filters%5Bteam%5D%5B%5D=40090"),
    ("Alpine", "2020", "https://www.motorsport.com/f1/news/?filters%5Bdiscovery_type%5D%5B%5D=1&filters%5Bchampionship%5D=2871&filters%5Bteam%5D%5B%5D=12080"),
    ("Alpine", "2020", "https://www.motorsport.com/f1/news/?filters%5Bdiscovery_type%5D%5B%5D=1&filters%5Bchampionship%5D=2871&filters%5Bteam%5D%5B%5D=784"),
    ("Alpine", "2020", "https://www.motorsport.com/f1/news/?filters%5Bdiscovery_type%5D%5B%5D=1&filters%5Bchampionship%5D=2871&filters%5Bteam%5D%5B%5D=182"),
    ("Red Bull Racing", "2020", "https://www.motorsport.com/f1/news/?filters%5Bdiscovery_type%5D%5B%5D=1&filters%5Bchampionship%5D=2871&filters%5Bteam%5D%5B%5D=1845"),
    ("Sauber", "2020", "https://www.motorsport.com/f1/news/?filters%5Bdiscovery_type%5D%5B%5D=1&filters%5Bchampionship%5D=2871&filters%5Bteam%5D%5B%5D=17888"),
    ("Ferrari", "2020", "https://www.motorsport.com/f1/news/?filters%5Bdiscovery_type%5D%5B%5D=1&filters%5Bchampionship%5D=2871&filters%5Bteam%5D%5B%5D=3"),
    ("HAAS", "2020", "https://www.motorsport.com/f1/news/?filters%5Bdiscovery_type%5D%5B%5D=1&filters%5Bchampionship%5D=2871&filters%5Bteam%5D%5B%5D=471"),
    ("Mercedes", "2020", "https://www.motorsport.com/f1/news/?filters%5Bdiscovery_type%5D%5B%5D=1&filters%5Bchampionship%5D=2871&filters%5Bteam%5D%5B%5D=5"),
    ("McLaren", "2020", "https://www.motorsport.com/f1/news/?filters%5Bdiscovery_type%5D%5B%5D=1&filters%5Bchampionship%5D=2871&filters%5Bteam%5D%5B%5D=10"),
    ("Red Bull Racing", "2020", "https://www.motorsport.com/f1/news/?filters%5Bdiscovery_type%5D%5B%5D=1&filters%5Bchampionship%5D=2871&filters%5Bteam%5D%5B%5D=4"),
    ("Racing Bulls", "2020", "https://www.motorsport.com/f1/news/?filters%5Bdiscovery_type%5D%5B%5D=1&filters%5Bchampionship%5D=2871&filters%5Bteam%5D%5B%5D=12"),
    ("Ferrari", "2020", "https://www.motorsport.com/f1/news/?filters%5Bdiscovery_type%5D%5B%5D=1&filters%5Bchampionship%5D=2871&filters%5Bteam%5D%5B%5D=40043"),
    ("Williams", "2020", "https://www.motorsport.com/f1/news/?filters%5Bdiscovery_type%5D%5B%5D=1&filters%5Bchampionship%5D=2871&filters%5Bteam%5D%5B%5D=13"),
    
    ("Alpine", "2021", "https://www.motorsport.com/f1/news/?filters%5Bdiscovery_type%5D%5B%5D=1&filters%5Bchampionship%5D=3115&filters%5Bteam%5D%5B%5D=20186"),
    ("Alpine", "2021", "https://www.motorsport.com/f1/news/?filters%5Bdiscovery_type%5D%5B%5D=1&filters%5Bchampionship%5D=3115&filters%5Bteam%5D%5B%5D=182"),
    ("Alpine", "2021", "https://www.motorsport.com/f1/news/?filters%5Bdiscovery_type%5D%5B%5D=1&filters%5Bchampionship%5D=3115&filters%5Bteam%5D%5B%5D=8"),
    ("Aston Martin", "2021", "https://www.motorsport.com/f1/news/?filters%5Bdiscovery_type%5D%5B%5D=1&filters%5Bchampionship%5D=3115&filters%5Bteam%5D%5B%5D=97"),
    ("Aston Martin", "2021", "https://www.motorsport.com/f1/news/?filters%5Bdiscovery_type%5D%5B%5D=1&filters%5Bchampionship%5D=3115&filters%5Bteam%5D%5B%5D=40090"),
    ("Ferrari", "2021", "https://www.motorsport.com/f1/news/?filters%5Bdiscovery_type%5D%5B%5D=1&filters%5Bchampionship%5D=3115&filters%5Bteam%5D%5B%5D=3"),
    ("Ferrari", "2021", "https://www.motorsport.com/f1/news/?filters%5Bdiscovery_type%5D%5B%5D=1&filters%5Bchampionship%5D=3115&filters%5Bteam%5D%5B%5D=40043"),
    ("HAAS", "2021", "https://www.motorsport.com/f1/news/?filters%5Bdiscovery_type%5D%5B%5D=1&filters%5Bchampionship%5D=3115&filters%5Bteam%5D%5B%5D=471"),
    ("Sauber", "2021", "https://www.motorsport.com/f1/news/?filters%5Bdiscovery_type%5D%5B%5D=1&filters%5Bchampionship%5D=3115&filters%5Bteam%5D%5B%5D=17888"),
    ("McLaren", "2021", "https://www.motorsport.com/f1/news/?filters%5Bdiscovery_type%5D%5B%5D=1&filters%5Bchampionship%5D=3115&filters%5Bteam%5D%5B%5D=10"),
    ("Mercedes", "2021", "https://www.motorsport.com/f1/news/?filters%5Bdiscovery_type%5D%5B%5D=1&filters%5Bchampionship%5D=3115&filters%5Bteam%5D%5B%5D=5"),
    ("Red Bull Racing", "2021", "https://www.motorsport.com/f1/news/?filters%5Bdiscovery_type%5D%5B%5D=1&filters%5Bchampionship%5D=3115&filters%5Bteam%5D%5B%5D=4"),
    ("Racing Bulls", "2021", "https://www.motorsport.com/f1/news/?filters%5Bdiscovery_type%5D%5B%5D=1&filters%5Bchampionship%5D=3115&filters%5Bteam%5D%5B%5D=12"),
    ("Williams", "2021", "https://www.motorsport.com/f1/news/?filters%5Bdiscovery_type%5D%5B%5D=1&filters%5Bchampionship%5D=3115&filters%5Bteam%5D%5B%5D=13"),

    ("Alpine", "2022", "https://www.motorsport.com/f1/news/?filters%5Bdiscovery_type%5D%5B%5D=1&filters%5Bchampionship%5D=4021&filters%5Bteam%5D%5B%5D=20186"),
    ("Aston Martin", "2022", "https://www.motorsport.com/f1/news/?filters%5Bdiscovery_type%5D%5B%5D=1&filters%5Bchampionship%5D=4021&filters%5Bteam%5D%5B%5D=97"),
    ("Ferrari", "2022", "https://www.motorsport.com/f1/news/?filters%5Bdiscovery_type%5D%5B%5D=1&filters%5Bchampionship%5D=4021&filters%5Bteam%5D%5B%5D=3"),
    ("Scuderia Ferrari", "2022", "https://www.motorsport.com/f1/news/?filters%5Bdiscovery_type%5D%5B%5D=1&filters%5Bchampionship%5D=4021&filters%5Bteam%5D%5B%5D=40043"),
    ("HAAS", "2022", "https://www.motorsport.com/f1/news/?filters%5Bdiscovery_type%5D%5B%5D=1&filters%5Bchampionship%5D=4021&filters%5Bteam%5D%5B%5D=471"),
    ("Sauber", "2022", "https://www.motorsport.com/f1/news/?filters%5Bdiscovery_type%5D%5B%5D=1&filters%5Bchampionship%5D=4021&filters%5Bteam%5D%5B%5D=9"),
    ("McLaren", "2022", "https://www.motorsport.com/f1/news/?filters%5Bdiscovery_type%5D%5B%5D=1&filters%5Bchampionship%5D=4021&filters%5Bteam%5D%5B%5D=10"),
    ("Mercedes", "2022", "https://www.motorsport.com/f1/news/?filters%5Bdiscovery_type%5D%5B%5D=1&filters%5Bchampionship%5D=4021&filters%5Bteam%5D%5B%5D=5"),
    ("Red Bull Racing", "2022", "https://www.motorsport.com/f1/news/?filters%5Bdiscovery_type%5D%5B%5D=1&filters%5Bchampionship%5D=4021&filters%5Bteam%5D%5B%5D=4"),
    ("Racing Bulls", "2022", "https://www.motorsport.com/f1/news/?filters%5Bdiscovery_type%5D%5B%5D=1&filters%5Bchampionship%5D=4021&filters%5Bteam%5D%5B%5D=12"),
    ("Alfa Romeo", "2022", "https://www.motorsport.com/f1/news/?filters%5Bdiscovery_type%5D%5B%5D=1&filters%5Bchampionship%5D=4021&filters%5Bteam%5D%5B%5D=17888"),
    ("Williams", "2022", "https://www.motorsport.com/f1/news/?filters%5Bdiscovery_type%5D%5B%5D=1&filters%5Bchampionship%5D=4021&filters%5Bteam%5D%5B%5D=13"),

    ("Alpine", "2023", "https://www.motorsport.com/f1/news/?filters%5Bdiscovery_type%5D%5B%5D=1&filters%5Bchampionship%5D=4446&filters%5Bteam%5D%5B%5D=20186"),
    ("Aston Martin", "2023", "https://www.motorsport.com/f1/news/?filters%5Bdiscovery_type%5D%5B%5D=1&filters%5Bchampionship%5D=4446&filters%5Bteam%5D%5B%5D=97"),
    ("Ferrari", "2023", "https://www.motorsport.com/f1/news/?filters%5Bdiscovery_type%5D%5B%5D=1&filters%5Bchampionship%5D=4446&filters%5Bteam%5D%5B%5D=3"),
    ("Scuderia Ferrari", "2023", "https://www.motorsport.com/f1/news/?filters%5Bdiscovery_type%5D%5B%5D=1&filters%5Bchampionship%5D=4446&filters%5Bteam%5D%5B%5D=40043"),
    ("HAAS", "2023", "https://www.motorsport.com/f1/news/?filters%5Bdiscovery_type%5D%5B%5D=1&filters%5Bchampionship%5D=4446&filters%5Bteam%5D%5B%5D=471"),
    ("Kick Sauber", "2023", "https://www.motorsport.com/f1/news/?filters%5Bdiscovery_type%5D%5B%5D=1&filters%5Bchampionship%5D=4446&filters%5Bteam%5D%5B%5D=9"),
    ("McLaren", "2023", "https://www.motorsport.com/f1/news/?filters%5Bdiscovery_type%5D%5B%5D=1&filters%5Bchampionship%5D=4446&filters%5Bteam%5D%5B%5D=10"),
    ("Mercedes", "2023", "https://www.motorsport.com/f1/news/?filters%5Bdiscovery_type%5D%5B%5D=1&filters%5Bchampionship%5D=4446&filters%5Bteam%5D%5B%5D=5"),
    ("Red Bull Racing", "2023", "https://www.motorsport.com/f1/news/?filters%5Bdiscovery_type%5D%5B%5D=1&filters%5Bchampionship%5D=4446&filters%5Bteam%5D%5B%5D=4"),
    ("Alfa Romeo", "2023", "https://www.motorsport.com/f1/news/?filters%5Bdiscovery_type%5D%5B%5D=1&filters%5Bchampionship%5D=4446&filters%5Bteam%5D%5B%5D=17888"),
    ("Racing Bulls", "2023", "https://www.motorsport.com/f1/news/?filters%5Bdiscovery_type%5D%5B%5D=1&filters%5Bchampionship%5D=4446&filters%5Bteam%5D%5B%5D=12"),
    ("Williams", "2023", "https://www.motorsport.com/f1/news/?filters%5Bdiscovery_type%5D%5B%5D=1&filters%5Bchampionship%5D=4446&filters%5Bteam%5D%5B%5D=13"),

    ("Alpine", "2024", "https://www.motorsport.com/f1/news/?filters%5Bchampionship%5D=4660&filters%5Bteam%5D%5B%5D=20186"),
    ("Aston Martin", "2024", "https://www.motorsport.com/f1/news/?filters%5Bchampionship%5D=4660&filters%5Bteam%5D%5B%5D=97"),
    ("Ferrari", "2024", "https://www.motorsport.com/f1/news/?filters%5Bchampionship%5D=4660&filters%5Bteam%5D%5B%5D=3"),
    ("Scuderia Ferrari", "2024", "https://www.motorsport.com/f1/news/?filters%5Bchampionship%5D=4660&filters%5Bteam%5D%5B%5D=40043"),
    ("HAAS", "2024", "https://www.motorsport.com/f1/news/?filters%5Bchampionship%5D=4660&filters%5Bteam%5D%5B%5D=471"),
    ("Kick Sauber", "2024", "https://www.motorsport.com/f1/news/?filters%5Bchampionship%5D=4660&filters%5Bteam%5D%5B%5D=9"),
    ("McLaren", "2024", "https://www.motorsport.com/f1/news/?filters%5Bchampionship%5D=4660&filters%5Bteam%5D%5B%5D=10"),
    ("Mercedes", "2024", "https://www.motorsport.com/f1/news/?filters%5Bchampionship%5D=4660&filters%5Bteam%5D%5B%5D=5"),
    ("Red Bull Racing", "2024", "https://www.motorsport.com/f1/news/?filters%5Bchampionship%5D=4660&filters%5Bteam%5D%5B%5D=4"),
    ("RB Racing", "2024", "https://www.motorsport.com/f1/news/?filters%5Bchampionship%5D=4660&filters%5Bteam%5D%5B%5D=5255"),
    ("Racing Bulls", "2024", "https://www.motorsport.com/f1/news/?filters%5Bchampionship%5D=4660&filters%5Bteam%5D%5B%5D=12"),
    ("Williams", "2024", "https://www.motorsport.com/f1/live-text/f1-abu-dhabi-gp-live-commentary-and-updates-race-1123853/1123854/"),

    ("Alpine", "2025", "https://www.motorsport.com/f1/news/?filters%5Bchampionship%5D=4855&filters%5Bteam%5D%5B%5D=20186"),
    ("Aston Martin", "2025", "https://www.motorsport.com/f1/news/?filters%5Bchampionship%5D=4855&filters%5Bteam%5D%5B%5D=97"),
    ("Ferrari", "2025", "https://www.motorsport.com/f1/news/?filters%5Bchampionship%5D=4855&filters%5Bteam%5D%5B%5D=3"),
    ("Scuderia Ferrari", "2025", "https://www.motorsport.com/f1/news/?filters%5Bchampionship%5D=4855&filters%5Bteam%5D%5B%5D=40043"),
    ("HAAS", "2025", "https://www.motorsport.com/f1/news/?filters%5Bchampionship%5D=4855&filters%5Bteam%5D%5B%5D=471"),
    ("Kick Sauber", "2025", "https://www.motorsport.com/f1/news/?filters%5Bchampionship%5D=4855&filters%5Bteam%5D%5B%5D=9"),
    ("McLaren", "2025", "https://www.motorsport.com/f1/news/?filters%5Bchampionship%5D=4855&filters%5Bteam%5D%5B%5D=10"),
    ("Mercedes", "2025", "https://www.motorsport.com/f1/news/?filters%5Bchampionship%5D=4855&filters%5Bteam%5D%5B%5D=5"),
    ("Red Bull Racing", "2025", "https://www.motorsport.com/f1/news/?filters%5Bchampionship%5D=4855&filters%5Bteam%5D%5B%5D=4"),
    ("Alfa Romeo", "2025", "https://www.motorsport.com/f1/news/?filters%5Bchampionship%5D=4855&filters%5Bteam%5D%5B%5D=17888"),
    ("Racing Bulls", "2025", "https://www.motorsport.com/f1/news/?filters%5Bchampionship%5D=4855&filters%5Bteam%5D%5B%5D=12"),
    ("Williams", "2025", "https://www.motorsport.com/f1/news/?filters%5Bchampionship%5D=4855&filters%5Bteam%5D%5B%5D=13")
]

In [10]:
def clean_extract_text(element):
    """
    Extracts text from a BS4 element.
    - If H1, H2, H3: Converts to UPPERCASE.
    - Adds '======' separator.
    """
    text = element.get_text(strip=True)
    if not text:
        return None
        
    tag = element.name
    if tag in ['h1', 'h2', 'h3']:
        text = text.upper()
        
    return f"{text}\n======"

def handle_popups(driver):
    """Attempts to close common cookie and ad popups to ensure content visibility."""
    # 1. Cookie Consent ("Accept all", "모두 허용", etc.)
    try:
        # Looking for buttons with specific text (case-insensitive)
        cookie_xpaths = [
            "//button[contains(translate(., 'ABCDEFGHIJKLMNOPQRSTUVWXYZ', 'abcdefghijklmnopqrstuvwxyz'), 'accept all')]",
            "//button[contains(., '모두 허용')]",
            "//a[contains(translate(., 'ABCDEFGHIJKLMNOPQRSTUVWXYZ', 'abcdefghijklmnopqrstuvwxyz'), 'accept all')]"
        ]
        
        for xpath in cookie_xpaths:
            btns = driver.find_elements(By.XPATH, xpath)
            for btn in btns:
                if btn.is_displayed():
                    btn.click()
                    print("INFO: 쿠키 동의 팝업 처리 완료.")
                    time.sleep(1) # Wait for popup to disappear
                    break
    except Exception:
        pass # It's okay if no popup is found

    # 2. Ad/Generic Popups ("close", "닫기")
    try:
        close_xpaths = [
            "//button[contains(translate(., 'ABCDEFGHIJKLMNOPQRSTUVWXYZ', 'abcdefghijklmnopqrstuvwxyz'), 'close')]",
            "//button[contains(., '닫기')]",
            "//div[contains(@class, 'close')]" # Common class for close buttons
        ]
        for xpath in close_xpaths:
            btns = driver.find_elements(By.XPATH, xpath)
            for btn in btns:
                 if btn.is_displayed():
                    btn.click()
                    print("INFO: 광고/안내 팝업 닫기 처리 완료.")
                    time.sleep(1)
                    break
    except Exception:
        pass

def analyze_team_mentions(text_lines, team_keys):
    """
    Analyzes the text content to find which team is mentioned most frequently.
    Returns: (best_matching_team_key, reason_string) or (None, reason_string)
    """
    # Combine all lines into one big string for easier searching
    full_text = " ".join(text_lines)
    
    counter = collections.Counter()
    
    # We search for the exact dictionary Key in the text.
    # Note: In a real scenario, you might want to map "Ferrari" -> "Scuderia_Ferrari", 
    # but the requirement says "search for the dictionary Keys".
    for team in team_keys:
        if team == "all": continue
        
        # Count occurrences of the Key string
        count = full_text.count(team) 
        
        # If no exact match on key (which might be long like 'Mercedes-AMG...'), 
        # fallback to a simpler partial check might be useful, but let's stick to the prompt:
        # "search for the dictionary Keys"
        
        # Let's try a slightly smarter search: replace underscores with spaces for the search
        readable_name = team.replace("_", " ")
        count += full_text.count(readable_name)
        
        if count > 0:
            counter[team] = count
            
    if not counter:
        return None, "No specific team names mentioned."
        
    most_common = counter.most_common()
    top_team, top_count = most_common[0]
    
    # Check for ties
    if len(most_common) > 1:
        second_team, second_count = most_common[1]
        if top_count == second_count:
            return None, f"Tie between {top_team} and {second_team} ({top_count} mentions each)."
            
    return top_team, f"Most mentions: {top_team} ({top_count})"


In [11]:
def main():
    print("=== F1 크롤러 시작 ===")
    
    # 1. Output Directory Creation
    try:
        os.makedirs(OUTPUT_DIR, exist_ok=True)
        print(f"INFO: 출력 폴더 '{OUTPUT_DIR}' 생성 성공.")
    except Exception as e:
        print(f"ERROR: 폴더 생성 실패: {e}")
        return

    # Buffer to hold text for each team: { 'TeamName': [ "Content 1", "Content 2" ] }
    team_data_buffers = collections.defaultdict(list)
    
    driver = setup_driver()
    all_team_keys = list(f1_teams.keys())
    
    try:
        for team_key, urls in f1_teams.items():
            for url in urls:
                try:
                    print(f"[{team_key}] Connecting to {url}...")
                    driver.get(url)
                    
                    # Wait for body to be present (basic load check)
                    try:
                        WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.TAG_NAME, "body")))
                    except Exception:
                        print(f"ERROR: [{team_key}] - [{url}] 타임아웃 또는 로딩 실패.")
                        continue
                        
                    # Handle Popups
                    handle_popups(driver)
                    
                    # Extract Content
                    soup = BeautifulSoup(driver.page_source, 'html.parser')
                    
                    # Extract h1, h2, h3, p in order
                    # We look for these tags anywhere in the body
                    # In many modern sites, content is in <main> or <article>. 
                    # If body is too noisy (navbars etc), we could narrow it down, 
                    # but 'h1, h2, h3, p' usually catches the article content well.
                    
                    target_tags = soup.find_all(['h1', 'h2', 'h3', 'p'])
                    
                    # Filter out empty or navigational/footer noise if possible
                    # Simple heuristic: ignore elements inside <nav>, <footer>, <header>
                    # But for now, let's stick to the prompt: "extract text contents of h1, h2, h3, p"
                    
                    extracted_lines_for_url = []
                    
                    for tag in target_tags:
                        # Skip if inside nav/footer/script/style (basic noise reduction)
                        if tag.find_parent(['nav', 'footer', 'header', 'script', 'style', 'noscript']):
                            continue
                            
                        formatted_text = clean_extract_text(tag)
                        if formatted_text:
                            extracted_lines_for_url.append(formatted_text)
                            
                    if not extracted_lines_for_url:
                        print(f"INFO: [{team_key}] - [{url}] 추출된 텍스트 없음.")
                        continue
                        
                    print(f"SUCCESS: [{team_key}] - [{url}] 크롤링 완료 ({len(extracted_lines_for_url)} 항목).")
                        
                    # Logic Distribution based on Key
                    target_team_file = team_key
                    
                    if team_key == 'all':
                        best_match, reason = analyze_team_mentions(extracted_lines_for_url, all_team_keys)
                        if best_match:
                            target_team_file = best_match
                            print(f"INFO: 'all' 링크 분석 결과 -> [{best_match}] 파일에 저장됩니다. ({reason})")
                        else:
                            # Fallback to 'all' instead of ignoring
                            target_team_file = "all"
                            print(f"INFO: 'all' 링크 분석 실패 또는 동률 -> 'all' 파일에 저장됩니다. ({reason})")
                            # Continue to add to buffer
                    
                    # Append to the target team's buffer
                    team_data_buffers[target_team_file].extend(extracted_lines_for_url)

                except Exception as e:
                    print(f"ERROR: [{team_key}] - [{url}] 처리 중 예외 발생: {e}")
                    
    finally:
        driver.quit()
        print("=== 크롤링 종료. 파일 저장 시작 ===")
        
    # Write Files
    for team, data_lines in team_data_buffers.items():
        if not data_lines:
            continue
            
        if team == "all":
            file_name = "all.txt"
        else:
            file_name = f"{team}.txt"
        file_path = os.path.join(OUTPUT_DIR, file_name)
        
        try:
            with open(file_path, "w", encoding="utf-8") as f:
                # Add a header or just write lines? Prompt says "extracted text sequentially"
                f.write("\n".join(data_lines))
            print(f"SUCCESS: [{team}] 데이터가 파일에 성공적으로 저장되었습니다.")
        except Exception as e:
            print(f"ERROR: [{team}] 파일 저장 실패: {e}")

if __name__ == "__main__":
    main()

=== F1 크롤러 시작 ===
INFO: 출력 폴더 '(ENG)F1_data' 생성 성공.


[all] Connecting to https://www.theguardian.com/sport/2023/mar/01/formula-one-2023-a-team-by-team-guide-to-the-cars-and-drivers...
SUCCESS: [all] - [https://www.theguardian.com/sport/2023/mar/01/formula-one-2023-a-team-by-team-guide-to-the-cars-and-drivers] 크롤링 완료 (59 항목).
INFO: 'all' 링크 분석 결과 -> [McLaren] 파일에 저장됩니다. (Most mentions: McLaren (6))
=== 크롤링 종료. 파일 저장 시작 ===
SUCCESS: [McLaren] 데이터가 파일에 성공적으로 저장되었습니다.
